# Project setup

In [1]:
import napari
import zarr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm
from DLC_for_WBFM.utils.video_and_data_conversion.import_video_as_array import get_single_volume
from DLC_for_WBFM.utils.projects.utils_project import load_config
from pathlib import Path
from DLC_for_WBFM.utils.projects.utils_project import safe_cd
from DLC_for_WBFM.utils.visualization.visualization_tracks import visualize_tracks
from DLC_for_WBFM.utils.preprocessing.utils_tif import PreprocessingSettings
from DLC_for_WBFM.utils.preprocessing.utils_tif import perform_preprocessing
from DLC_for_WBFM.utils.preprocessing.utils_tif import preprocess_all_frames_using_config
import os
from magicgui import magicgui

%load_ext autoreload
%autoreload 2

In C:\Users\charles.fieseler\Anaconda3\envs\DLC-for-WBFM\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\charles.fieseler\Anaconda3\envs\DLC-for-WBFM\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\charles.fieseler\Anaconda3\envs\DLC-for-WBFM\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\charles.fieseler\Anaconda3\envs\DLC-for-WBFM\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed tw

In [2]:
# Set up project

project_path = r"Y:\shared_projects\wbfm\dlc_stacks\Charlie-immobilized-long\project_config.yaml"
cfg = load_config(project_path)

num_z = cfg['dataset_params']['num_slices']

project_dir = Path(project_path).parent

with safe_cd(project_dir):
    seg_fname = Path(cfg['subfolder_configs']['segmentation'])
    seg_cfg = dict(load_config(seg_fname))
    
    train_fname = Path(cfg['subfolder_configs']['training_data'])
    train_cfg = dict(load_config(train_fname))
    
    track_fname = Path(cfg['subfolder_configs']['tracking'])
    track_cfg = dict(load_config(track_fname))
    
    p_fname = train_cfg['preprocessing_config']
    p = PreprocessingSettings.load_from_yaml(p_fname)
    
    cfg['preprocessing_config'] = p_fname

In [3]:
# Import segmentation
fname = seg_cfg['output']['masks']
with safe_cd(project_dir):
    raw_segmentation = zarr.open(fname)

fname = os.path.join('4-traces', 'reindexed_masks.zarr')
with safe_cd(project_dir):
    reindexed_segmentation = zarr.open(fname)

In [21]:
# Import DLC tracks
fname = track_cfg['final_3d_tracks']['df_fname']
with safe_cd(project_dir):
    df = pd.read_hdf(fname)

In [5]:
# Import raw data
fname = cfg['preprocessed_red']
with safe_cd(project_dir):
    red_data = zarr.open(fname)

# Define the GUI and layers

In [6]:
def build_df_of_current_points(viewer: napari.Viewer, neuron_names: list) -> pd.DataFrame:
    new_points = viewer.layers['pts_with_past'].data

    col = pd.MultiIndex.from_product([neuron_names, ['t', 'z', 'x', 'y']])
    df_new = pd.DataFrame(columns=col)

    df_new[(name, 't')] = new_points[:,0]
    df_new[(name, 'z')] = new_points[:,1]
    df_new[(name, 'y')] = new_points[:,2]
    df_new[(name, 'x')] = new_points[:,3]

    df_new.sort_values((name,'t'), inplace=True, ignore_index=True)

    return df_new


def save_df(df, name: str) -> None:
    # Save
    out_fname = os.path.join(project_dir, '3-tracking', 'manual_tracking', f'{name}.h5')
    # df_new.to_hdf(out_fname)

    out_fname = str(Path(out_fname).with_suffix('.csv'))
#     df_old = pd.read_csv(out_fname)
#     df_old[name] = df_new[name]
#     df_old.to_csv(out_fname, mode='a')
    df_new.to_csv(out_fname)  # Just overwrite
    
    print(f"Saved manual annotations for neuron {name} at {out_fname}")

def build_tracks_from_name(name='neuron0'):
    # Just visualize one neuron for now
    # 5 columns: 
    # track_id, t, z, y, x
    coords = ['z', 'y', 'x']
    all_tracks_list = []
    likelihood_thresh = 0.4
    zxy_array = np.array(df[name][coords])
    t_array = np.expand_dims(np.arange(zxy_array.shape[0]), axis=1)
    # Remove low likelihood
    to_keep = df[name]['likelihood'] > likelihood_thresh
    zxy_array = zxy_array[to_keep,:]
    t_array = t_array[to_keep,:]

    all_tracks_list.append(np.hstack([t_array, zxy_array]))

    return np.vstack(all_tracks_list)

In [11]:
# neuron_names = list(df.columns.levels[0])

# @magicgui(call_button='Save Annotations',
#          name={"choices": neuron_names})
# def save_widget(viewer: napari.Viewer, name='neuron0') -> napari.types.LayerDataTuple:
#     df = build_df_of_current_points(viewer, neuron_names)
#     save_df(df, name)
    
#     track_array = build_tracks_from_name(name)
    
#     return (track_array, {'name': 'pts_with_future_and_past'}, 'points')

In [12]:
# w1 = widgets.PushButton(value=True, text='PushButton.text')
# w2 = widgets.CheckBox(value=False, text='CheckBox.text')
# container = widgets.Container(widgets=[w1, w2])
# container.show()

In [13]:
def zoom_using_viewer(viewer: napari.Viewer) -> None:
    # Get current point
    t = viewer.dims.current_step[0]
    tzxy = viewer.layers['pts_with_past'].data[t]
    
    # Zoom to it in XY
    viewer.camera.zoom = 10
    viewer.camera.center = tzxy[1:]
    
    # Zoom in Z
    viewer.dims.current_step = (t, tzxy[1], 0, 0)

    
def change_viewer_time_point(viewer: napari.Viewer, dt: int) -> None:
    # Increment time
    t = viewer.dims.current_step[0] + dt
    tzxy = (t, ) + viewer.dims.current_step[1:]
    viewer.dims.current_step = tzxy
    
    
@magicgui(call_button='Zoom to current point')
def zoom_widget(viewer: napari.Viewer) -> None:
    zoom_using_viewer(viewer)

In [14]:
@magicgui(call_button='Zoom to next point')
def zoom_next_widget(viewer: napari.Viewer) -> None:
    change_viewer_time_point(viewer, 1)
    zoom_using_viewer(viewer)

@magicgui(call_button='Zoom to previous point')
def zoom_previous_widget(viewer: napari.Viewer) -> None:
    change_viewer_time_point(viewer, -1)
    zoom_using_viewer(viewer)

In [15]:
type(zoom_next_widget)

magicgui.widgets._function_gui.FunctionGui

In [16]:
all_tracks_array = build_tracks_from_name('neuron0')
pts_with_future_and_past = all_tracks_array
# pts_only_one_frame = pts_with_past # Not made to edit; only 2d version
points_opt = dict(face_color='blue', size=4)

In [17]:
track_of_point = np.hstack([np.ones((all_tracks_array.shape[0], 1)), all_tracks_array])

# Try 2 for GUI: just make it in QT5

In [6]:
from PyQt5 import QtCore, QtWidgets

In [7]:
 viewer = napari.view_image(red_data, ndisplay=2, opacity=0.5)
# viewer = napari.view_image(reindexed_segmentation, ndisplay=2, opacity=0.5)
# viewer.add_labels(reindexed_segmentation, visible=False)
# viewer.add_labels(raw_segmentation, visible=False)
# viewer.add_points(pts_with_future_and_past, n_dimensional=True, symbol='square', **points_opt)
# viewer.add_points(pts_only_one_frame, edge_color='red', symbol='star', visible=False, **points_opt)
# viewer.add_tracks(track_of_point)

In [8]:
# viewer.add_points(ndim=3, name = "pts_with_future_and_past", n_dimensional=True, symbol='square', **points_opt)
# viewer.add_tracks(np.zeros((1,5)), name = "track_of_point")

In [9]:
# def zoom_using_viewer(viewer: napari.Viewer) -> None:
#     # Get current point
#     t = viewer.dims.current_step[0]
#     tzxy = viewer.layers['pts_with_future_and_past'].data[t]
    
#     # Zoom to it in XY
#     viewer.camera.zoom = 10
#     viewer.camera.center = tzxy[1:]
    
#     # Zoom in Z
#     viewer.dims.current_step = (t, tzxy[1], 0, 0)

In [10]:
from DLC_for_WBFM.gui.utils.manual_annotation import manual_annotation_widget

In [22]:
# Make the widget with my new class and add
main_window = viewer.window._qt_window
output_dir = r"Y:\shared_projects\wbfm\dlc_stacks\Charlie-immobilized-long\3-tracking\manual_tracking"

ui = manual_annotation_widget()
# ui.setupUi(main_window, df, output_dir, viewer)
ui.setupUi(df, output_dir, viewer)

# Actually dock
viewer.window.add_dock_widget(ui)
ui.show()

# @viewer.bind_key('.', overwrite=True)
# def zoom_next(viewer: napari.Viewer) -> None:
#     change_viewer_time_point(viewer, 1)
#     zoom_using_viewer(viewer)

# @viewer.bind_key(',', overwrite=True)
# def zoom_previous(viewer: napari.Viewer) -> None:
#     change_viewer_time_point(viewer, -1)
#     zoom_using_viewer(viewer)

     neuron0                                       
           t     z          x          y likelihood
0        0.0  17.0  45.357874  71.882004   0.997100
1        1.0  17.0  45.356854  71.880191   0.996462
2        2.0  17.0  45.393682  71.893656   0.996453
3        3.0  17.0  45.371265  71.897385   0.996421
4        4.0  17.0  45.515534  71.853137   0.995701
...      ...   ...        ...        ...        ...
4495  4495.0  17.0  62.498827  71.339777   0.884234
4496  4496.0   0.0   0.000000   0.000000   0.000000
4497  4497.0   0.0   0.000000   0.000000   0.000000
4498  4498.0  17.0  62.580890  71.383208   0.886086
4499  4499.0  17.0  62.746904  71.336567   0.910911

[4500 rows x 5 columns]
     neuron1                                         
           t     z           x           y likelihood
0        0.0  18.0  218.575275  101.605244   0.998570
1        1.0  18.0  218.574101  101.583130   0.998650
2        2.0  18.0  218.621160  101.539171   0.998522
3        3.0  18.0  218.71448

# Use the GUI to track

In [44]:
viewer = napari.view_image(red_data, ndisplay=2, opacity=0.5)
# viewer = napari.view_image(reindexed_segmentation, ndisplay=2, opacity=0.5)
# viewer.add_labels(reindexed_segmentation, visible=False)
# viewer.add_labels(raw_segmentation, visible=False)
viewer.add_points(pts_with_future_and_past, n_dimensional=True, symbol='square', **points_opt)
# viewer.add_points(pts_only_one_frame, edge_color='red', symbol='star', visible=False, **points_opt)
viewer.add_tracks(track_of_point)

<Tracks layer 'track_of_point' at 0x27fdeaefbc8>

In [45]:
viewer.window.add_dock_widget(save_widget)
# viewer.window.add_dock_widget(zoom_widget)
# viewer.window.add_dock_widget(zoom_next_widget)

In [13]:
@viewer.bind_key('.', overwrite=True)
def zoom_next(viewer: napari.Viewer) -> None:
    change_viewer_time_point(viewer, 1)
    zoom_using_viewer(viewer)

In [14]:
@viewer.bind_key(',', overwrite=True)
def zoom_previous(viewer: napari.Viewer) -> None:
    change_viewer_time_point(viewer, -1)
    zoom_using_viewer(viewer)

In [25]:
viewer.title

'napari'

In [16]:
viewer.layers['pts_with_past'].data[10]

array([10.        , 17.        , 71.86739407, 45.45017191])

In [17]:
viewer.dims.current_step

(0, 0, 0, 0)

In [48]:
main_window = viewerwindow._qt_window

In [54]:
viewer.layers['track_of_point'].data

array([[1.00000000e+00, 0.00000000e+00, 1.70000000e+01, 8.78200378e+00,
        1.23578739e+01],
       [1.00000000e+00, 1.00000000e+00, 1.70000000e+01, 7.18801910e+01,
        4.53568541e+01],
       [1.00000000e+00, 2.00000000e+00, 1.70000000e+01, 7.18936564e+01,
        4.53936824e+01],
       ...,
       [1.00000000e+00, 4.49500000e+03, 1.70000000e+01, 7.13397774e+01,
        6.24988270e+01],
       [1.00000000e+00, 4.49800000e+03, 1.70000000e+01, 7.13832081e+01,
        6.25808895e+01],
       [1.00000000e+00, 4.49900000e+03, 1.70000000e+01, 7.13365667e+01,
        6.27469041e+01]])

# Next: just look at training data

In [2]:
from DLC_for_WBFM.utils.visualization.utils_segmentation import get_or_recalculate_which_frames
from DLC_for_WBFM.gui.utils.manual_annotation import manual_annotation_widget

In [3]:
# Set up project

project_path = r"Y:\shared_projects\wbfm\dlc_stacks\Charlie-worm3-new-seg\project_config.yaml"
cfg = load_config(project_path)

num_z = cfg['dataset_params']['num_slices']

project_dir = Path(project_path).parent

with safe_cd(project_dir):
    seg_fname = Path(cfg['subfolder_configs']['segmentation'])
    segment_cfg = dict(load_config(seg_fname))
    
    train_fname = Path(cfg['subfolder_configs']['training_data'])
    train_cfg = dict(load_config(train_fname))
    
    track_fname = Path(cfg['subfolder_configs']['tracking'])
    track_cfg = dict(load_config(track_fname))

this_config = {'track_cfg':track_cfg, 'segment_cfg': segment_cfg, 'project_cfg':cfg}
this_config['dataset_params'] = cfg['dataset_params'].copy()

DEBUG = False


with safe_cd(project_dir):
    fname = os.path.join('2-training_data', 'raw', 'clust_df_dat.pickle')
    df = pd.read_pickle(fname)

    # Get the frames chosen as training data, or recalculate
    which_frames = list(get_or_recalculate_which_frames(DEBUG, df, this_config))

In [4]:
# Import segmentation
fname = segment_cfg['output']['masks']
with safe_cd(project_dir):
    raw_segmentation = zarr.open(fname)

fname = os.path.join('2-training_data', 'training_data_tracks.h5')
with safe_cd(project_dir):
    df = pd.read_hdf(fname)

In [5]:
# Import raw data
fname = cfg['preprocessed_red']
with safe_cd(project_dir):
    red_data = zarr.open(fname)

In [6]:
viewer = napari.view_image(red_data[which_frames[0]:which_frames[-1],...], ndisplay=2, opacity=0.5)


In [8]:
viewer.add_labels(raw_segmentation[which_frames[0]:which_frames[-1],...])

<Labels layer 'Labels' at 0x255bc649f88>

In [7]:
main_window = viewer.window._qt_window
output_dir = r"Y:\shared_projects\wbfm\dlc_stacks\Charlie-immobilized-long\2-training_data\manual_tracking"

ui = manual_annotation_widget()
ui.setupUi(df['Charlie'], output_dir, viewer)

# Actually dock
viewer.window.add_dock_widget(ui)
ui.show()

Unable to load numpy_formathandler accelerator from OpenGL_accelerate


# Aside: convert my dataframe format to DLC style (training data)

In [4]:
from DLC_for_WBFM.utils.visualization.utils_segmentation import get_or_recalculate_which_frames
from DLC_for_WBFM.utils.visualization.visualize_using_dlc import build_subset_df
from DLC_for_WBFM.utils.training_data.tracklet_to_DLC import convert_training_dataframe_to_dlc_format

In [5]:
project_path = r"Y:\shared_projects\wbfm\dlc_stacks\Charlie-worm3-new-seg\project_config.yaml"

project_cfg = load_config(project_path)
project_dir = Path(project_path).parent

with safe_cd(project_dir):
    track_fname = Path(project_cfg['subfolder_configs']['tracking'])
    track_cfg = dict(load_config(track_fname))

    segment_fname = Path(project_cfg['subfolder_configs']['segmentation'])
    segment_cfg = dict(load_config(segment_fname))

this_config = {'track_cfg':track_cfg, 'segment_cfg': segment_cfg, 'project_cfg':project_cfg}
this_config['dataset_params'] = project_cfg['dataset_params'].copy()

DEBUG = False

# Import raw data
fname = project_cfg['preprocessed_red']
with safe_cd(project_dir):
    red_data = zarr.open(fname)

In [6]:

with safe_cd(project_dir):
    fname = os.path.join('2-training_data', 'raw', 'clust_df_dat.pickle')
    df = pd.read_pickle(fname)

    # Get the frames chosen as training data, or recalculate
    which_frames = list(get_or_recalculate_which_frames(DEBUG, df, this_config))

    # Build a sub-df with only the relevant neurons; all slices
    # Todo: connect up to actually tracked z slices?
    subset_opt = {'which_z': None,
                  'max_z_dist': None,
                  'verbose': 1}
    subset_df = build_subset_df(df, which_frames, **subset_opt)

74 tracklets overlap in time
Keeping 74/15160 tracklets


In [9]:
training_df = convert_training_dataframe_to_dlc_format(subset_df, scorer='Charlie')

In [11]:
# TEMPORARY
out_fname = r"Y:\shared_projects\wbfm\dlc_stacks\Charlie-worm3-new-seg\2-training_data\training_data_tracks.h5"
training_df.to_hdf(out_fname, 'df_with_missing')

In [10]:
training_df

scorer       Charlie                                                \
bodyparts   neuron67                                    neuron1550   
coords             z           x           y likelihood          z   
527        14.473324  478.440492  301.878249        1.0   8.433333   
528        14.439147  442.296110  297.927227        1.0   8.444166   
529        14.464619  434.457944  295.923899        1.0   8.444020   
530        14.449612  432.425065  285.927649        1.0   8.433498   
531        14.463445  440.394052  273.902107        1.0   8.435424   
532        14.466667  450.434667  259.908000        1.0   8.432331   
533        14.415502  462.312579  259.927573        1.0   8.425879   
534        14.444444  458.346720  249.938420        1.0   8.369792   
535        16.410439  454.234875  253.937129        1.0   8.437870   
536        16.433985  452.392421  253.920538        1.0   8.424674   
537        14.448231  446.364351  241.917431        1.0   8.416357   
538        16.416567  450.307323  229.926771        1.0   8.405141   
539        16.429091  462.345455  235.884848        1.0   8.427518   
540        16.450355  468.268322  235.918440        1.0   8.426683   
541        16.424051  474.345570  247.936709        1.0   8.441943   
542        16.434783  482.263285  269.951691        1.0   8.420786   
543        16.425216  488.310260  271.904821        1.0   8.425926   
544        16.432500  484.351250  281.977500        1.0   8.434010   
545        16.445867  454.245634  287.941793        1.0   8.452174   
546        14.456258  444.348587  293.881561        1.0   6.422283   
547        14.444010  428.391927  299.950521        1.0   8.437500   
548        14.432927  432.271951  293.971951        1.0   8.430303   
549        14.433121  428.420382  279.945223        1.0   8.450450   
550        14.401949  436.211937  271.947625        1.0   8.440154   
551        14.428743  452.265868  263.953293        1.0   8.415601   
552        14.422843  464.263670  257.964763        1.0   8.416667   
553        14.444444  458.374339  251.966931        1.0   8.384309   
554        14.428758  452.351634  247.964706        1.0   8.439901   
555        16.406404  454.257389  249.947044        1.0   8.421302   
556        14.428571  442.347763  243.906205        1.0   6.412518   
557        16.419847  446.361323  243.958015        1.0   6.403657   

scorer                                                               ...  \
bodyparts                                    neuron1673              ...   
coords              x           y likelihood          z           x  ...   
527        466.282716  199.979012        1.0  12.349057  426.281671  ...   
528        446.358846  193.938519        1.0  14.164740  414.364162  ...   
529        430.326972  193.914758        1.0  14.313144  400.188144  ...   
530        404.282020  183.934729        1.0  14.236842  372.457064  ...   
531        388.246002  181.948339        1.0  12.414305  354.288799  ...   
532        378.276942  187.906015        1.0  12.427966  344.334746  ...   
533        370.295226  211.929648        1.0  12.384286  336.262857  ...   
534        356.226562  222.028646        1.0  12.392130  326.322931  ...   
535        354.250888  237.924260        1.0  12.398385  328.304172  ...   
536        354.205219  255.937129        1.0  12.397112  332.186522  ...   
537        358.273854  279.934325        1.0  12.398658  346.312752  ...   
538        356.232558  273.944920        1.0  12.399751  348.276463  ...   
539        362.287469  259.896806        1.0  10.443878  352.376276  ...   
540        366.280048  233.914663        1.0  10.414268  350.379224  ...   
541        380.233412  209.922986        1.0  10.397959  352.376276  ...   
542        408.354880  195.953105        1.0  10.374202  370.324393  ...   
543        440.264198  181.985185        1.0  12.349810  400.193916  ...   
544        460.314721  183.904822        1.0  10.408673  416.377135  ...   
545        464.320497  